# Clean up the Knowledge Graph

This notebook deletes all the nodes, edges, indexes and constraints in the Neo4j graph database!!!! This makes sure you have a fresh database to start with.

## Import Libraries

In [41]:
import os 
import configparser

In [42]:
# install or import Neo4j GraphDataScience library
try: 
  from graphdatascience import GraphDataScience
  print('Successfully imported GraphDataScience')
except ModuleNotFoundError:
  !pip3 install graphdatascience
  from graphdatascience import GraphDataScience
  print('installed and imported GraphDataScience')

Successfully imported GraphDataScience


# Custom Functions

In [43]:
# function adapted from Neo4j GDS Demo Notebook (h/t Zach B.) mainly used for connecting to Neo4j database from Python
def read_neo4j_properties(NEO4J_PROPERTIES_FILE: str=None) -> str:
  '''Parses Neo4j database or Aura connection details from provided .ini filepath.
  Requirements:
    configparser

  Args:
    NEO4J_PROPERTIES_FILE: path to a .ini file
  
  Returns:
    HOST: link to Neo4j or Aura host 
    USERNAME: login username
    PASSWORD: login password 

  Note: The .ini file should use the following syntax
    [NEO4J]
    PASSWORD=<password>
    USERNAME=<database name>
    HOST=<host uri>

  If no path is passed, the function will return the defaults:
    HOST = 'neo4j://localhost'
    USERNAME = 'neo4j'
    PASSWORD = 'password'
  '''

  if NEO4J_PROPERTIES_FILE is not None and os.path.exists(NEO4J_PROPERTIES_FILE):
      config = configparser.RawConfigParser()
      config.read(NEO4J_PROPERTIES_FILE)
      HOST = config['NEO4J']['HOST']
      USERNAME = config['NEO4J']['USERNAME']
      PASSWORD = config['NEO4J']['PASSWORD']
      print('Using HOST, USERNAME, PASSWORD from .ini file')
      return HOST, USERNAME, PASSWORD
  else:
      print('Could not find database properties file, using defaults:')
      HOST = 'neo4j://localhost'
      USERNAME = 'neo4j'
      PASSWORD = 'password'
      print(f'HOST: {HOST} \nUSERHAME: {USERNAME} \nPASSWORD: {PASSWORD}')
      return HOST, USERNAME, PASSWORD 

# Connect to Neo4j DB
It is recommended to store authentication credentials in a separate file and read them in to the notebook as variables. This code assumes the files are stored in a local auth directory.

In [44]:
# get authentication credentials from local auth file
NEO4J_PROPERTIES_FILE = 'auth/immerse_kg_auth.ini'
HOST, USERNAME, PASSWORD = read_neo4j_properties(NEO4J_PROPERTIES_FILE=NEO4J_PROPERTIES_FILE)

Using HOST, USERNAME, PASSWORD from .ini file


In [ ]:
# connect to neo4j instance 
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=False)

In [46]:
# confirm connection with GDS version 
gds.version()

'2.5.0'

# Delete all the nodes and edges

In [47]:
gds.run_cypher('''
                MATCH(n) 
                DETACH DELETE n
                ''')

""


# Delete all the indexes and constraints

In [48]:
gds.run_cypher('''
                CALL apoc.schema.assert({}, {})
                ''')

,label,key,keys,unique,action
0,DictionaryConcept,name,[name],False,DROPPED
1,TextMining,None,"[base, head, np, doc_id, term, type]",False,DROPPED
2,BookChapter,None,"[isbm, title, authors, year, publisher]",False,DROPPED
3,OntologicalConcept,name,[name],False,DROPPED
4,BookChapter,isbn,[isbn],True,DROPPED
5,OntologicalConcept,id,[id],True,DROPPED
6,DictionaryConcept,id,[id],True,DROPPED
7,TextMining,id,[id],True,DROPPED
